In [1]:
# transformers for huggingface modes
# datasets for huggingface datasets
# bitsandbytes for quantization
!pip install transformers torch -q
!pip install --upgrade datasets -q
!pip install bitsandbytes --no-cache-dir -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not

In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, BitsAndBytesConfig

In [3]:
# Using the app_reviews dataset, which has a rating and review.
# Rating would be used to create labelled data for sentiment analysis
# sentiment analysis is being used to pick the top positive and negative reviews based on prediction probability.
dataset = load_dataset("sealuzh/app_reviews")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.42k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/288065 [00:00<?, ? examples/s]

In [4]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['package_name', 'review', 'date', 'star'],
        num_rows: 288065
    })
})

In [6]:
# converting to pandas
df = dataset["train"].to_pandas()

In [7]:
df.head()

,package_name,review,date,star
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5


In [8]:
# Checking the number of reviews per app
df.package_name.value_counts()

,count
package_name,
com.google.android.gms,103535
org.telegram.messenger,22816
org.ppsspp.ppsspp,9771
com.frostwire.android,8867
com.google.android.marvin.talkback,7807
...,...
com.moonpi.swiftnotes,12
com.nathanosman.chronosnap,11
net.ralphbroenink.muzei.unsplash,11


<h4> Review Classification </h4>

In [9]:
# Converting Rating to a sentiment score to check with RoBERTa predictions
# If the rating is more than 3, then we are labelling as positive. If less than 3, then negative.
# df["sentiment"] = df["star"].apply(lambda x: 1 if x > 3 else 0)

In [10]:
# Checking the review length to decide on the kind of RoBERTa model to be used
df["review_len"] =df["review"].apply(len)

In [11]:
# The maximum length of reviews is 2335 characters.
df["review_len"].describe()

,review_len
count,288065.000000
mean,63.297304
std,93.477861
min,1.000000
25%,10.000000
50%,26.000000
75%,77.000000
max,2335.000000


In [12]:
df.head()

,package_name,review,date,star,sentiment,review_len
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4,1,290
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4,1,141
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5,1,131
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3,0,345
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5,1,79


In [13]:
df.columns

Index(['package_name', 'review', 'date', 'star', 'sentiment', 'review_len'], dtype='object')

In [14]:
# Checking if GPU is available. If so, setting the device to GPU. Or else, using CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
# Loading the cardiffnlp version which was finetuned on twitter data for sentiment classification
# Load model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model_sentiment = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [16]:
# Move model to GPU
model_sentiment.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [17]:
# Defining labels. The labels for RoBERTa are in this order only
labels = ["Negative", "Neutral", "Positive"]

In [18]:
def analyze_sentiment(sentences):
    results = []
    for sentence in tqdm(sentences, desc="Processing Reviews"):
        # Tokenize input, truncate to 512 characters, and use padding
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

        # Perform inference
        with torch.no_grad():
            # **inputs is used to pass both input_ids and attention mask
            outputs = model_sentiment(**inputs)
            # model outputs the logits. The order is always Negative, Neutral, and Positive
            logits = outputs.logits
            # The last dimension stores the logits, hence dim=-1
            probs = torch.softmax(logits, dim=-1)

        # Extract sentiment
        predicted_label = torch.argmax(probs, dim=-1).item()
        results.append((labels[predicted_label], probs[0].cpu().numpy()))
    return results

In [19]:
# Storing the sentiment and probability computed by RoBERTa
results = analyze_sentiment(df["review"])
df["sentiment"] = [result[0] for result in results]
df["sentiment_probs"] = [result[1] for result in results]

Processing Reviews: 100%|██████████| 288065/288065 [44:04<00:00, 108.91it/s]


In [20]:
df.head(100)

,package_name,review,date,star,sentiment,review_len,sentiment_probs
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4,Positive,290,"[0.0019521475, 0.02050587, 0.9775419]"
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4,Positive,141,"[0.122359104, 0.24866298, 0.6289779]"
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5,Positive,131,"[0.003940717, 0.06713458, 0.9289247]"
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3,Negative,345,"[0.4991507, 0.45113, 0.04971926]"
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5,Positive,79,"[0.002189858, 0.031223401, 0.9665867]"
...,...,...,...,...,...,...,...
95,rs.pedjaapps.alogcatroot.app,Description about Internet access is a lie. Th...,January 17 2017,1,Negative,246,"[0.8344414, 0.14655371, 0.01900487]"
96,rs.pedjaapps.alogcatroot.app,It is very helpful,March 29 2017,5,Positive,18,"[0.002815577, 0.045515884, 0.95166856]"
97,com.iskrembilen.quasseldroid,Crippling power consumption The devs really ne...,October 27 2016,4,Negative,194,"[0.87303996, 0.10967216, 0.017287873]"
98,com.iskrembilen.quasseldroid,Unusable in large rooms No way to filter out j...,October 23 2016,2,Negative,60,"[0.7540967, 0.23365572, 0.012247576]"


In [21]:
# The model is performing well. The reviews aren't negative though the stars are
df[(df["star"] < 3) & (df["sentiment"] != 'Negative')]

,package_name,review,date,star,sentiment,review_len,sentiment_probs
81,rs.pedjaapps.alogcatroot.app,Logcat Its good help,July 30 2016,1,Positive,20,"[0.0057259835, 0.111314304, 0.88295966]"
87,rs.pedjaapps.alogcatroot.app,Unresponsive maybe some threads are in order. ...,August 29 2015,2,Neutral,67,"[0.28885958, 0.5665973, 0.14454316]"
90,rs.pedjaapps.alogcatroot.app,Internet permission Unnecessary internet permi...,October 17 2014,1,Neutral,51,"[0.3931212, 0.56297594, 0.04390289]"
116,com.iskrembilen.quasseldroid,Outdated Why isn't quasseldroid updated on the...,October 23 2015,2,Neutral,149,"[0.36948168, 0.57919705, 0.051321246]"
121,com.iskrembilen.quasseldroid,Play version outdated The version on F-Droid i...,May 08 2015,2,Neutral,54,"[0.07432208, 0.8636695, 0.062008444]"
...,...,...,...,...,...,...,...
287955,org.geometerplus.zlibrary.ui.android,it has limited app not free ask for paid app i...,March 16 2017,1,Neutral,78,"[0.36586478, 0.5783656, 0.055769622]"
287956,org.geometerplus.zlibrary.ui.android,La mejor que puede existir..,March 16 2017,1,Neutral,28,"[0.037562363, 0.70114356, 0.26129404]"
287960,org.geometerplus.zlibrary.ui.android,It can not change font size for Write to left ...,March 18 2017,1,Neutral,87,"[0.47887388, 0.49221838, 0.028907796]"
288018,org.geometerplus.zlibrary.ui.android,Hangs regularly on my device.,April 15 2017,2,Neutral,29,"[0.06030929, 0.8764563, 0.06323441]"


In [22]:
# The results seem okay.
df[(df["star"] > 3) & (df["sentiment"] != 'Positive')]

,package_name,review,date,star,sentiment,review_len,sentiment_probs
7,com.mantz_it.rfanalyzer,Works with RTL and Nextbook Aries 8. Demod sto...,May 19 2016,5,Neutral,104,"[0.41873375, 0.53286636, 0.048399884]"
16,com.mantz_it.rfanalyzer,Nice Tested with Rtl SDR. Working fine althoug...,February 20 2016,4,Neutral,180,"[0.26225176, 0.40975714, 0.32799107]"
18,com.mantz_it.rfanalyzer,Sweet Pick your source device in settings. Def...,February 01 2016,5,Neutral,85,"[0.07245515, 0.8353216, 0.09222319]"
20,com.mantz_it.rfanalyzer,The app looks good but does not work so far wi...,January 31 2016,4,Negative,167,"[0.766495, 0.206654, 0.026850957]"
22,com.mantz_it.rfanalyzer,A God send! Awful interference problem with my...,January 09 2016,5,Negative,231,"[0.8871205, 0.102063544, 0.010816041]"
...,...,...,...,...,...,...,...
288039,org.geometerplus.zlibrary.ui.android,Will you ever add Reflow?,April 28 2017,5,Neutral,25,"[0.04086489, 0.9097118, 0.0494233]"
288040,org.geometerplus.zlibrary.ui.android,Was unable to follow URL in a footnote.,April 27 2017,4,Negative,39,"[0.70991987, 0.27959132, 0.01048887]"
288042,org.geometerplus.zlibrary.ui.android,Like,April 26 2017,5,Neutral,4,"[0.20353667, 0.5713448, 0.22511853]"
288057,com.termux.api,how to install additional package,July 08 2016,4,Neutral,33,"[0.11418974, 0.8315447, 0.054265555]"


In [23]:
# Sentiment is getting predicted correctly.
df[(df["star"] > 3) & (df["sentiment"] == 'Positive')]

,package_name,review,date,star,sentiment,review_len,sentiment_probs
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4,Positive,290,"[0.0019521475, 0.02050587, 0.9775419]"
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4,Positive,141,"[0.122359104, 0.24866298, 0.6289779]"
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5,Positive,131,"[0.003940717, 0.06713458, 0.9289247]"
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5,Positive,79,"[0.002189858, 0.031223401, 0.9665867]"
5,com.mantz_it.rfanalyzer,Good job Tried a few . This has to be the most...,July 19 2016,5,Positive,222,"[0.026077392, 0.24615149, 0.7277711]"
...,...,...,...,...,...,...,...
288058,com.termux.api,Fantastic. The perfect add on for termux if yo...,July 06 2016,5,Positive,88,"[0.0028593189, 0.04755086, 0.9495898]"
288061,com.termux.api,I like this app . Its is very helpful for use....,June 20 2016,5,Positive,49,"[0.0013153906, 0.011462023, 0.9872226]"
288062,com.termux.api,Finally Brings back the Unix command line to A...,May 20 2016,5,Positive,53,"[0.008704583, 0.46992686, 0.5213685]"
288063,com.termux.api,The API feature is great just need loads more...,May 05 2016,5,Positive,139,"[0.00434378, 0.024672154, 0.97098404]"


In [24]:
# Sentiment is getting predicted correctly.
df[(df["star"] < 3) & (df["sentiment"] == 'Negative')]

,package_name,review,date,star,sentiment,review_len,sentiment_probs
10,com.mantz_it.rfanalyzer,I'll forgo the refund. But no go with Watson d...,March 31 2016,1,Negative,115,"[0.9654278, 0.030641112, 0.003931038]"
13,com.mantz_it.rfanalyzer,Crash when set a demodulation! :(,February 22 2016,2,Negative,33,"[0.8658252, 0.121054344, 0.013120371]"
17,com.mantz_it.rfanalyzer,doesn't work with rtl-2832 Says it works with ...,February 10 2016,1,Negative,220,"[0.8761823, 0.10942366, 0.014393973]"
38,com.mantz_it.rfanalyzer,Do I need to hack my phone for this applicatio...,August 01 2015,1,Negative,55,"[0.68011963, 0.30651775, 0.013362515]"
41,com.mantz_it.rfanalyzer,Does not work Did not work with my rtl-sdr,May 23 2015,1,Negative,42,"[0.93570006, 0.06043948, 0.0038604734]"
...,...,...,...,...,...,...,...
288036,org.geometerplus.zlibrary.ui.android,It sucks,April 24 2017,1,Negative,8,"[0.9494764, 0.04281183, 0.0077117854]"
288043,org.geometerplus.zlibrary.ui.android,Poor outdated inter phase does not format tab...,April 26 2017,2,Negative,86,"[0.8903243, 0.10078551, 0.008890193]"
288048,org.geometerplus.zlibrary.ui.android,Can't get app permission to access books and I...,May 02 2017,1,Negative,71,"[0.96137077, 0.036081053, 0.0025482047]"
288052,com.termux.api,Not working,August 30 2016,1,Negative,11,"[0.8544341, 0.13413113, 0.011434773]"


In [25]:
df.sentiment.value_counts()

,count
sentiment,
Positive,161165
Neutral,72673
Negative,54227


In [26]:
# Exporting results to a CSV
df.to_csv("SentimentAnalysisResults.csv", index=False)

In [27]:
df.groupby(['package_name', 'sentiment'], as_index=False)["review"].count()

,package_name,sentiment,review
0,a2dp.Vol,Negative,13
1,a2dp.Vol,Neutral,11
2,a2dp.Vol,Positive,35
3,acr.browser.lightning,Negative,88
4,acr.browser.lightning,Neutral,34
...,...,...,...
1165,vu.de.urpool.quickdroid,Neutral,4
1166,vu.de.urpool.quickdroid,Positive,21
1167,wb.receiptspro,Negative,12
1168,wb.receiptspro,Neutral,19


In [28]:
# Collecting only positive or negative reviews for Summarization
dfPolarReviews = df[df.sentiment != "Neutral"]

<h4> Review Summarization </h4>

In [29]:
# Group by package_name and sentiment, and count the reviews
dfBothReviews = dfPolarReviews.groupby(['package_name', 'sentiment'], as_index=False)["review"].count()

# Pivot the data to create separate columns for positive and negative review counts
dfPivot = dfBothReviews.pivot(index='package_name', columns='sentiment', values='review').fillna(0)

# Filter products with both positive and negative reviews
dfFiltered = dfPivot[(dfPivot.get('Positive', 0) > 0) & (dfPivot.get('Negative', 0) > 0)]

# Reset index
dfFiltered = dfFiltered.reset_index()

In [30]:
# 387 products have both positive and negative reviews.
dfFiltered.shape[0]

387

In [31]:
dfFiltered.head()

sentiment,package_name,Negative,Positive
0,a2dp.Vol,13.0,35.0
1,acr.browser.lightning,88.0,172.0
2,app.easytoken,2.0,16.0
3,app.openconnect,18.0,110.0
4,apps.droidnotify,13.0,47.0


In [32]:
# Separating out the probabilities to identify the most polar reviews
dfPolarReviews[['negative_prob', 'neutral_prob', 'positive_prob']] = pd.DataFrame(dfPolarReviews['sentiment_probs'].tolist(), index=dfPolarReviews.index)

<ipython-input-32-62bc35c46adb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPolarReviews[['negative_prob', 'neutral_prob', 'positive_prob']] = pd.DataFrame(dfPolarReviews['sentiment_probs'].tolist(), index=dfPolarReviews.index)
<ipython-input-32-62bc35c46adb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPolarReviews[['negative_prob', 'neutral_prob', 'positive_prob']] = pd.DataFrame(dfPolarReviews['sentiment_probs'].tolist(), index=dfPolarReviews.index)
<ipython-input-32-62bc35c46adb>:2: Setting

In [33]:
dfPolarReviews.head()

,package_name,review,date,star,sentiment,review_len,sentiment_probs,negative_prob,neutral_prob,positive_prob
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4,Positive,290,"[0.0019521475, 0.02050587, 0.9775419]",0.001952,0.020506,0.977542
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4,Positive,141,"[0.122359104, 0.24866298, 0.6289779]",0.122359,0.248663,0.628978
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5,Positive,131,"[0.003940717, 0.06713458, 0.9289247]",0.003941,0.067135,0.928925
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3,Negative,345,"[0.4991507, 0.45113, 0.04971926]",0.499151,0.451130,0.049719
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5,Positive,79,"[0.002189858, 0.031223401, 0.9665867]",0.002190,0.031223,0.966587


In [34]:
dfPolarReviews['predicted_polarity'] = dfPolarReviews[['positive_prob', 'negative_prob', 'neutral_prob']].idxmax(axis=1)

<ipython-input-34-d08d0ef4fef7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPolarReviews['predicted_polarity'] = dfPolarReviews[['positive_prob', 'negative_prob', 'neutral_prob']].idxmax(axis=1)


In [35]:
# Sorting based on polarity and review length to get the most polar longest review.
dfPolarReviews = dfPolarReviews.sort_values(by=['package_name', 'predicted_polarity', 'review_len'], ascending=[True, True, False])

In [36]:
top_reviews = (
    dfPolarReviews.groupby(['package_name', 'predicted_polarity'])
    .head(10)
    .reset_index(drop=True)
)

In [51]:
# Aggregating reviews together as review number followed by the review followed by two line breaks

# agg_reviews = (
#     top_reviews
#     .groupby(['package_name', 'sentiment'], as_index=False)
#     .agg({'review': lambda texts: "\n\n".join(texts)})
# )
agg_reviews = (
    top_reviews
    .groupby(['package_name', 'sentiment'], as_index=False)
    .agg({
        'review': lambda texts: "\n\n".join(
            [f"Review {i+1}: {text}" for i, text in enumerate(texts)]
        )
    })
)

In [52]:
agg_reviews.head()

,package_name,sentiment,review
0,a2dp.Vol,Negative,Review 1: The app worked great but.... ... it...
1,a2dp.Vol,Positive,Review 1: Been looking for something to launch...
2,acr.browser.lightning,Negative,Review 1: Great Browser needs an update Updat...
3,acr.browser.lightning,Positive,Review 1: Very impressive browser and I love t...
4,app.easytoken,Negative,Review 1: Just one token 4stars for the moment...


In [53]:
# Checking the length of the reviews to decide on the strategy.
agg_reviews["review_length"] = agg_reviews["review"].apply(len)

In [54]:
agg_reviews['review_length'].describe()

,review_length
count,782.000000
mean,2892.873402
std,1900.727835
min,21.000000
25%,1372.750000
50%,2591.500000
75%,4201.750000
max,11495.000000


In [42]:
# Using the bart-large-cnn model finetuned on summarizing CNN articles
from transformers import BartForConditionalGeneration, BartTokenizer

# Load model and tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [43]:
def generate_summary_with_bart(package_name, polarity, reviews_text):
    # Construct the prompt
    prompt = (
        f"Summarize the following reviews about the app '{package_name}' into a concise paragraph. "
        f"Use bullets to highlight the {polarity} aspects :\n"
        f"Reviews: \n\n{reviews_text}"
    )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=200,
        min_length=50,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [44]:
# Trying out a single example
package_name = "xyz"
polarity = "negative"
review_text = """Been looking for something to launch and connect my phone to my Acura TLX. Tried a few then found this. Overall it works but when connected to th car it takes over your phone so if I need to get into phone the hime button does not work I have to hit the back button and apps button. If it did not make the phone difficult to use while running this app is great but it is too frustrating to get to email or any other app while this is running and connected to car. July 2016 update: Upped the rating because I was in car mode. The app is pretty good although the disconnect wifi feature is not working. If this app works for you use it but I found one a tad more reliable. Excellent app that works very well. Totally free with no ads. Running Moto G3 with marshmallow 6.0 this app works very well. It has separate volume presets for media alarm notif and call. It toggles the volumes when the Bluetooth is connected or disconnected to allow you to have special settings to work with your device and return to normal status afterwards. This helped me to improve weak call volume settings in my speakerphone . As a bonus this app even turns on Pandora. Note that this is not a volume booster. It can only go to max volume. It can't go to 11. Exactly what I needed and free! Bluetooth has been a nightmare on my Nexus 6 since the beginning. Recent changes with Android 6.0.1 made it even messier. Never know what you're going to get with volumes. There were also connectivity delays and volume holdovers from other multimedia apps. This app clears all that up! I can set volume by device and it is specifically set up for use in the car which was the number one use for me. I can't thank the developers enough! Thanks so much! A++ This seems almost perfect for me. Using the ""voice call stream"" it interrupts the radio (Ford Sync) as a hands free call and reads the text/message so no missing them when I am not listening to the ""aux"" mode of the radio where the ""music stream"" is usually heard. The only problem I have is that the ""call"" sometimes does not end after reading the message. I have to manually ""hang up"" (not the end of the world!). Great app though. Thank you. Different volumes for different devices! If you want to have all your stuff have consistent volumes without constantly adjusting the settings for your car stereo headphones Millennium Falcon bike speakers etc without fiddling with the settings in PowerAmp and everything else this works. Don't forget to hit save and give a few options a try; it does work and it's awesome. AWESOME! Well when it works that is... Well thought out but not supported anymore. Works some of the time. Does not start up on boot. Will launch an app of choice bit sometimes won't change the volume when connected to a Bluetooth device. Like go find an updated version if this? Solves Android 6.0.1 Bluetooth volume problems Not sure I need all the clever things to do with multiple Bluetooth devices but by intercepting Bluetooth headset volume calls to Android 6.0.1 on my Nexus 6P you've solved the volume problems at a stroke. Thank you. On my rooted Nexus 6p with Nougat 7.1.1 it solves the most annoying problem in my first world cupboard of problems - my volume was being reset to 50% every time I connected to a bluetooth media device. This app solves that be automatically raising the volume. Excellent app. Been using it for years on many different phones to automatically read my text messages though my cars Bluetooth voice stream. Just wish it could direct my google maps voice commands through the voice stream as well. Hopefully some day. I got this app mainly to read text messages through my headset but it will not stay connected most of the time to read them. When it's connected it works great. I will probably end up uninstalling it if I can't get it to stay connected
"""
output = generate_summary_with_bart(package_name, polarity, review_text)
print(output)

The app is pretty good although the disconnect wifi feature is not working. Solves Android 6.0.1 Bluetooth volume problems. Works some of the time. Does not start up on boot. Will launch an app of choice bit sometimes won't change the volume when connected to a Bluetooth device.


In [45]:
output

"The app is pretty good although the disconnect wifi feature is not working. Solves Android 6.0.1 Bluetooth volume problems. Works some of the time. Does not start up on boot. Will launch an app of choice bit sometimes won't change the volume when connected to a Bluetooth device."

In [46]:
agg_reviews_test = agg_reviews.head(10)

In [47]:
tqdm.pandas()

agg_reviews_test["summary"] = agg_reviews_test.progress_apply(
    lambda row: generate_summary_with_bart(row["package_name"], row["sentiment"], row["review"]),
    axis=1
)

100%|██████████| 10/10 [00:10<00:00,  1.08s/it]
<ipython-input-47-2aa44be93920>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_reviews_test["summary"] = agg_reviews_test.progress_apply(


In [48]:
agg_reviews_test.head()

,package_name,sentiment,review,review_length,summary
0,a2dp.Vol,Negative,Review 1: The app worked great but.... ... it...,1332,Reviews on the app have been mostly negative. ...
1,a2dp.Vol,Positive,Review 1: Been looking for something to launch...,3978,The app connects your phone to your car using ...
2,acr.browser.lightning,Negative,Review 1: Great Browser needs an update Updat...,4447,The browser crashed at least twice a day. It c...
3,acr.browser.lightning,Positive,Review 1: Very impressive browser and I love t...,6456,Lightning is the best Android browser I've eve...
4,app.easytoken,Negative,Review 1: Just one token 4stars for the moment...,289,Summarize the following reviews about the app ...


In [55]:
# Applying on entire dataframe.
tqdm.pandas()

agg_reviews["summary"] = agg_reviews.progress_apply(
    lambda row: generate_summary_with_bart(row["package_name"], row["sentiment"], row["review"]),
    axis=1
)

100%|██████████| 782/782 [15:04<00:00,  1.16s/it]


In [56]:
agg_reviews.to_csv("ApplicationReviewSummary.csv", index=False)